## 线性回归实战（sklearn + 评估）
### 1.加载 Boston 数据集（或  替代）
### 2.划分训练/测试集，标准化特征（StandardScaler）
### 3.用 LinearRegression 训练模型
### 4.计算 MSE、RMSE、R²，并解释含义
### 5.可视化：预测值 vs 真实值散点图

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

SEED = 42
# 加载数据
all_house_train_data = pd.read_csv('../data/house_train.csv')
all_house_test_data = pd.read_csv('../data/house_test.csv')

# 数据处理
# print(f'数据的特征含义:{all_house_train_data.head()}')
# print(f'房屋数据信息:{all_house_train_data.info()}')

# 计算缺失比例超过50%的列，原理先使用all_house_train_data.isnull()生成布尔值，空值为true，非空的值为false，再通过计算每列的平均数，如果
# 平均数大于0.5则证明出现true，空值的数量的比例大于50%，有些时候处理数据时候需要去除缺失比例超过50%的列，可用此方法筛选出
loss_data = all_house_train_data.isnull().mean()[all_house_train_data.isnull().mean() > 0.5]

# 合并特征表（为了一次性做相同的预处理）
# 目的：保证独热编码后的列在训练/测试上完全一致。
# 训练集去掉 Id 和 SalePrice；测试集去掉 Id，iloc是左闭右开的形式
# pd.concat中的axis=0是行从上到下拼接，axis=1是列的从左到右拼接
all_features = pd.concat([all_house_train_data.iloc[:, 1:-1], all_house_test_data.iloc[:, 1:]],ignore_index=True,axis=0)

# 数值特征缺失值填充 & 标准化
# 目的：让不同量纲的数值特征处在可比尺度上；先填缺失再标准化更稳健。
# 找出数值列（排除object）
numeric_cols = all_features.dtypes[all_features.dtypes != 'object'].index
# 用每列均值填充数值缺失
all_features[numeric_cols] = all_features[numeric_cols].fillna(all_features[numeric_cols].mean())
# Z-score 标准化,不同量纲的特征（如“年龄”和“收入”）被缩放到可比的尺度
all_features[numeric_cols] = all_features[numeric_cols].apply(lambda x: (x - x.mean())/(x.std()))
# 类别特征独热编码
# 目的：把字符串类别变成 0/1 特征；保留 float（不要整表转 int，否则会破坏数值列的小数）。
all_features = pd.get_dummies(all_features, dummy_na=True)

# 拆回训练/测试 + 提取标签
# 目的：把合并后的特征再分回去；同时拿出训练集标签 SalePrice。
n_train = all_house_train_data.shape[0]   #shape返回数据的格式为（行数,列数）这里取0代表获取训练数据的行数
train_features = all_features.iloc[:n_train].values.astype(np.float32)   # 提取训练数据
test_features  = all_features.iloc[n_train:].values.astype(np.float32)  # 提取测试数据
train_labels_raw = all_house_train_data['SalePrice'].values.astype(np.float32) # 提取训练标签

# 切分训练集/验证集
# 目的：从训练集再划出一部分做验证，监控过拟合。
'''
    train_features: 训练数据，不包含ID和数据头
    train_labels_raw：需要训练的标签，比如说这里就是最终要预测的房价
    test_size：训练集内部划分测试集和训练集的比例
    random_state：随机数种子，确保结果可复现
    返回：
    函数返回 4 个对象（如果只传入 X，则返回 2 个）：
    X_train: 分割出的训练集特征(训练的数据)。
    X_test（X_valid）: 分割出的测试集特征(测试的数据)。
    y_train: 分割出的训练集标签(需要得出预测的列)。
    y_test（y_valid）: 分割出的测试集标签。
'''
X_train, X_valid, y_train_raw, y_valid_raw = train_test_split(train_features, train_labels_raw, test_size=0.2, random_state=SEED)
# print(X_train.shape, X_valid.shape)
# 进行模型训练
class func:
    def __init__(self):
        self.w = None
        self.b = None
    def fix(self,train_features,train_labels_raw):
        train_features = np.concat([np.ones((train_features.shape[0],1)),train_features],axis=1)
        XTX = train_features.T @ train_features
        XTy = train_features.T @ train_labels_raw

        theta = np.linalg.lstsq(XTX) @ XTy
        print(theta)


linearRegressio = func()
linearRegressio.fix(X_train,y_train_raw)


AttributeError: module 'numpy' has no attribute 'concat'

In [1]:
# 第2天：线性回归实战（sklearn + 评估）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. 加载数据集
print("正在加载 California Housing 数据集...")
data = fetch_california_housing()
X, y = data.data, data.target

# 查看数据基本信息
print(f"特征数量: {X.shape[1]}")
print(f"样本数量: {X.shape[0]}")
print(f"特征名称: {data.feature_names}")
print(f"目标变量（房价中位数，单位：万美元）范围: [{y.min():.2f}, {y.max():.2f}]")

# 转为 DataFrame（可选，便于查看）
df = pd.DataFrame(X, columns=data.feature_names)
df['MedHouseVal'] = y
# print(df.head())

# 2. 划分训练集和测试集（80% 训练，20% 测试）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. 特征标准化（对线性模型很重要！）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # 注意：只用训练集的均值和方差

# 4. 训练线性回归模型
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# 5. 预测
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# 6. 评估模型性能
def evaluate(y_true, y_pred, label=""):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\n{label} 评估结果:")
    print(f"  MSE  : {mse:.4f}")
    print(f"  RMSE : {rmse:.4f} (万美元)")
    print(f"  MAE  : {mae:.4f} (万美元)")
    print(f"  R²   : {r2:.4f}")
    return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2}

train_metrics = evaluate(y_train, y_train_pred, "训练集")
test_metrics = evaluate(y_test, y_test_pred, "测试集")

# 7. 可视化：真实值 vs 预测值（测试集）
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_test_pred, alpha=0.6, color='steelblue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("真实房价（万美元）")
plt.ylabel("预测房价（万美元）")
plt.title("线性回归：真实值 vs 预测值（测试集）")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

# 8. 【Bonus】查看每个特征的系数（解释模型）
feature_importance = pd.DataFrame({
    'Feature': data.feature_names,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

print("\n特征系数（绝对值排序）:")
print(feature_importance)

# 可视化特征重要性（系数大小）
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Coefficient'])
plt.xlabel("线性回归系数")
plt.title("各特征对房价的线性影响（正：正相关，负：负相关）")
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

正在加载 California Housing 数据集...


/home/jiangchengxuan/anaconda3/envs/d2l/lib/python3.13/site-packages/sklearn/datasets/_base.py:1518: UserWarning: Retry downloading from url: https://ndownloader.figshare.com/files/5976036
  warnings.warn(f"Retry downloading from url: {remote.url}")


URLError: <urlopen error [Errno 111] Connection refused>